### TOC – Instancia Baja (Valparaíso)

In [ ]:

import numpy as np
import random
import statistics
import time
from tqdm import tqdm

distancias = np.array([
    [5, 8, 11],
    [7, 4, 10],
    [6, 6, 6],
    [9, 5, 7],
    [8, 9, 5],
    [4, 7, 12]
])
costos_fijos = np.array([20, 25, 15])
capacidades = np.array([2, 3, 2])
D_MAX = 8
n_clients = distancias.shape[0]
n_hubs = distancias.shape[1]
dim = n_clients

def evaluar(asignacion):
    conteo = [0] * n_hubs
    usado = [0] * n_hubs
    costo = 0
    for c, j in enumerate(asignacion):
        if j < 0 or j >= n_hubs or distancias[c][j] > D_MAX:
            return float('inf')
        conteo[j] += 1
        usado[j] = 1
        if conteo[j] > capacidades[j]:
            return float('inf')
        costo += distancias[c][j]
    for j in range(n_hubs):
        if usado[j]:
            costo += costos_fijos[j]
    return costo


POP_SIZE = 20
MAX_ITERS = 30
NT = 3  # Thunderstorms
NO = 1  # Tornado
DIM = n_clients


def generar_individuo_factible():
    individuo = []
    for c in range(n_clients):
        opciones = [j for j in range(n_hubs) if distancias[c][j] <= D_MAX]
        if opciones:
            individuo.append(random.choice(opciones))
        else:
            return None
    return individuo

def init_population():
    poblacion = []
    while len(poblacion) < POP_SIZE:
        ind = generar_individuo_factible()
        if ind is not None and evaluar(ind) < float('inf'):
            poblacion.append(np.array(ind))
    return poblacion

def clip(sol):
    return np.clip(sol, 0, n_hubs - 1).astype(int)

def toc_optimizer():
    pop = init_population()
    fitness = [evaluar(ind) for ind in pop]

    for t in range(MAX_ITERS):
        indices_sorted = np.argsort(fitness)
        pop = [pop[i] for i in indices_sorted]
        fitness = [fitness[i] for i in indices_sorted]

        tornado = pop[0]
        thunderstorms = pop[1:NT + 1]
        windstorms = pop[NT + 1:]

        new_pop = [tornado.copy()] + [ts.copy() for ts in thunderstorms]

        for w in windstorms:
            leader = random.choice([tornado] + thunderstorms)
            new_w = w.copy()
            for i in range(DIM):
                opciones = [j for j in range(n_hubs) if distancias[i][j] <= D_MAX]
                if not opciones:
                    new_w[i] = -1
                elif random.random() < 0.5:
                    diff = leader[i] - w[i]
                    move = int(np.sign(diff)) if diff != 0 else 0
                    candidato = w[i] + move
                    new_w[i] = candidato if candidato in opciones else random.choice(opciones)
                else:
                    new_w[i] = random.choice(opciones)

            new_w = clip(new_w)
            if -1 in new_w or evaluar(new_w) == float('inf'):
                new_pop.append(w)
            elif evaluar(new_w) < evaluar(w):
                new_pop.append(new_w)
            else:
                new_pop.append(w)

        pop = new_pop
        fitness = [evaluar(ind) for ind in pop]

    best_fit = min(fitness)
    best_sol = pop[np.argmin(fitness)]
    return best_fit, best_sol

print(" Ejecutando TOC Algorithm - Instancia Baja (30 corridas)...")
print("=" * 50)
print(f" Problema: {n_clients} clientes, {n_hubs} hubs")
print(f" Capacidades por hub: {list(capacidades)}")
print(f" Distancia máxima: {D_MAX}")
print(f" Costos fijos: {list(costos_fijos)}")

tiempo_inicio = time.time()

resultados = []
mejores_soluciones = []
tiempos_individuales = []

print("\nRESULTADOS POR CORRIDA:")
print("-" * 30)

for corrida in range(30):
    # Medir tiempo de cada corrida individual
    inicio_corrida = time.time()
    val, sol = toc_optimizer()
    fin_corrida = time.time()

    tiempo_corrida = fin_corrida - inicio_corrida
    tiempos_individuales.append(tiempo_corrida)

    resultados.append(val)
    mejores_soluciones.append(sol)

    print(f"Corrida {corrida + 1:2d}: {val}")

    if (corrida + 1) % 10 == 0:
        print(f"Completadas {corrida + 1}/30 corridas... (Tiempo promedio por corrida: {np.mean(tiempos_individuales):.3f}s)")

# Tiempo total transcurrido
tiempo_fin = time.time()
tiempo_total = tiempo_fin - tiempo_inicio

print("\n RESULTADOS TOC - INSTANCIA BAJA:")
print("=" * 50)
print(f" Mejor resultado:        {min(resultados)}")
print(f" Peor resultado:         {max(resultados)}")
print(f" Promedio:               {np.mean(resultados):.2f}")
print(f" Mediana:                {np.median(resultados)}")
print(f" Desviación estándar:    {np.std(resultados):.2f}")
print(f" Rango intercuartílico:  {np.percentile(resultados, 75) - np.percentile(resultados, 25):.2f}")

print(f"\n⏱  TIEMPOS DE EJECUCIÓN:")
print("=" * 50)
print(f" Tiempo total:           {tiempo_total:.3f} segundos")
print(f" Tiempo promedio/corrida: {np.mean(tiempos_individuales):.3f} segundos")
print(f" Tiempo más rápido:      {min(tiempos_individuales):.3f} segundos")
print(f" Tiempo más lento:       {max(tiempos_individuales):.3f} segundos")
print(f" Desv. std tiempo:       {np.std(tiempos_individuales):.3f} segundos")

# Mostrar la mejor solución encontrada
mejor_indice = np.argmin(resultados)
mejor_solucion = mejores_soluciones[mejor_indice]
print(f"\nMEJOR SOLUCIÓN ENCONTRADA (Costo: {min(resultados)}):")
print("=" * 50)
print("Cliente → Hub asignado:")
for i, hub in enumerate(mejor_solucion):
    print(f"Cliente {i+1:2d} → Hub {hub+1} (distancia: {distancias[i][hub]})")

# Verificar factibilidad de la mejor solución
print(f"\n🔍 VERIFICACIÓN DE LA MEJOR SOLUCIÓN:")
print("=" * 50)
conteo_hubs = [0] * n_hubs
for cliente, hub in enumerate(mejor_solucion):
    conteo_hubs[hub] += 1

print("Ocupación de hubs:")
costo_fijo_total = 0
for j in range(n_hubs):
    if conteo_hubs[j] > 0:
        print(f"Hub {j+1}: {conteo_hubs[j]}/{capacidades[j]} clientes - Costo fijo: {costos_fijos[j]}")
        costo_fijo_total += costos_fijos[j]
    else:
        print(f"Hub {j+1}: No usado (capacidad {capacidades[j]})")

costo_distancias = sum(distancias[i][hub] for i, hub in enumerate(mejor_solucion))
print(f"\n Costo total distancias: {costo_distancias}")
print(f" Costo total fijo:       {costo_fijo_total}")
print(f" COSTO TOTAL:            {costo_distancias + costo_fijo_total}")



### Código TOC – Instancia MEDIA

In [ ]:
import numpy as np
import random
import statistics
import time
from tqdm import tqdm

distancias = np.array([
    [6, 9, 12, 7, 8],
    [5, 8, 6, 10, 9],
    [8, 7, 9, 6, 11],
    [7, 5, 6, 9, 10],
    [6, 9, 8, 6, 9],
    [5, 4, 5, 8, 8],
    [8, 7, 9, 6, 9],
    [6, 6, 7, 8, 7],
    [5, 9, 10, 7, 11],
    [7, 6, 7, 9, 8],
    [6, 5, 8, 9, 10],
    [8, 6, 9, 10, 6]
])
costos_fijos = np.array([20, 25, 18, 22, 19])
capacidades = np.array([3, 3, 3, 3, 3])
D_MAX = 8
n_clients = distancias.shape[0]
n_hubs = distancias.shape[1]
dim = n_clients

def evaluar(asignacion):
    conteo = [0] * n_hubs
    usado = [0] * n_hubs
    costo = 0
    for c, j in enumerate(asignacion):
        if j < 0 or j >= n_hubs or distancias[c][j] > D_MAX:
            return float('inf')
        conteo[j] += 1
        usado[j] = 1
        if conteo[j] > capacidades[j]:
            return float('inf')
        costo += distancias[c][j]
    for j in range(n_hubs):
        if usado[j]:
            costo += costos_fijos[j]
    return costo


POP_SIZE = 20
MAX_ITERS = 30
NT = 3  # Thunderstorms
NO = 1  # Tornado
DIM = n_clients


def generar_individuo_factible():
    individuo = []
    for c in range(n_clients):
        opciones = [j for j in range(n_hubs) if distancias[c][j] <= D_MAX]
        if opciones:
            individuo.append(random.choice(opciones))
        else:
            return None
    return individuo

def init_population():
    poblacion = []
    while len(poblacion) < POP_SIZE:
        ind = generar_individuo_factible()
        if ind is not None and evaluar(ind) < float('inf'):
            poblacion.append(np.array(ind))
    return poblacion

def clip(sol):
    return np.clip(sol, 0, n_hubs - 1).astype(int)

def toc_optimizer():
    pop = init_population()
    fitness = [evaluar(ind) for ind in pop]

    for t in range(MAX_ITERS):
        indices_sorted = np.argsort(fitness)
        pop = [pop[i] for i in indices_sorted]
        fitness = [fitness[i] for i in indices_sorted]

        tornado = pop[0]
        thunderstorms = pop[1:NT + 1]
        windstorms = pop[NT + 1:]

        new_pop = [tornado.copy()] + [ts.copy() for ts in thunderstorms]

        for w in windstorms:
            leader = random.choice([tornado] + thunderstorms)
            new_w = w.copy()
            for i in range(DIM):
                opciones = [j for j in range(n_hubs) if distancias[i][j] <= D_MAX]
                if not opciones:
                    new_w[i] = -1
                elif random.random() < 0.5:
                    diff = leader[i] - w[i]
                    move = int(np.sign(diff)) if diff != 0 else 0
                    candidato = w[i] + move
                    new_w[i] = candidato if candidato in opciones else random.choice(opciones)
                else:
                    new_w[i] = random.choice(opciones)

            new_w = clip(new_w)
            if -1 in new_w or evaluar(new_w) == float('inf'):
                new_pop.append(w)
            elif evaluar(new_w) < evaluar(w):
                new_pop.append(new_w)
            else:
                new_pop.append(w)

        pop = new_pop
        fitness = [evaluar(ind) for ind in pop]

    best_fit = min(fitness)
    best_sol = pop[np.argmin(fitness)]
    return best_fit, best_sol


print(" Ejecutando TOC Algorithm - Instancia Media (30 corridas)...")
print("=" * 50)
print(f" Problema: {n_clients} clientes, {n_hubs} hubs")
print(f" Capacidad por hub: {capacidades[0]}")
print(f" Distancia máxima: {D_MAX}")


tiempo_inicio = time.time()

resultados = []
mejores_soluciones = []
tiempos_individuales = []

print("\nRESULTADOS POR CORRIDA:")
print("-" * 30)

for corrida in range(30):

    inicio_corrida = time.time()
    val, sol = toc_optimizer()
    fin_corrida = time.time()

    tiempo_corrida = fin_corrida - inicio_corrida
    tiempos_individuales.append(tiempo_corrida)

    resultados.append(val)
    mejores_soluciones.append(sol)

    print(f"Corrida {corrida + 1:2d}: {val}")

    if (corrida + 1) % 10 == 0:
        print(f"Completadas {corrida + 1}/30 corridas... (Tiempo promedio por corrida: {np.mean(tiempos_individuales):.3f}s)")


tiempo_fin = time.time()
tiempo_total = tiempo_fin - tiempo_inicio

print("\n RESULTADOS TOC - INSTANCIA MEDIA:")
print("=" * 50)
print(f" Mejor resultado:        {min(resultados)}")
print(f" Peor resultado:         {max(resultados)}")
print(f" Promedio:               {np.mean(resultados):.2f}")
print(f" Mediana:                {np.median(resultados)}")
print(f" Desviación estándar:    {np.std(resultados):.2f}")
print(f" Rango intercuartílico:  {np.percentile(resultados, 75) - np.percentile(resultados, 25):.2f}")

print(f"\n⏱  TIEMPOS DE EJECUCIÓN:")
print("=" * 50)
print(f" Tiempo total:           {tiempo_total:.3f} segundos")
print(f" Tiempo promedio/corrida: {np.mean(tiempos_individuales):.3f} segundos")
print(f" Tiempo más rápido:      {min(tiempos_individuales):.3f} segundos")
print(f" Tiempo más lento:       {max(tiempos_individuales):.3f} segundos")
print(f" Desv. std tiempo:       {np.std(tiempos_individuales):.3f} segundos")


mejor_indice = np.argmin(resultados)
mejor_solucion = mejores_soluciones[mejor_indice]
print(f"\n🏆 MEJOR SOLUCIÓN ENCONTRADA (Costo: {min(resultados)}):")
print("=" * 50)
print("Cliente → Hub asignado:")
for i, hub in enumerate(mejor_solucion):
    print(f"Cliente {i+1:2d} → Hub {hub+1} (distancia: {distancias[i][hub]})")


print(f"\n🔍 VERIFICACIÓN DE LA MEJOR SOLUCIÓN:")
print("=" * 50)
conteo_hubs = [0] * n_hubs
for cliente, hub in enumerate(mejor_solucion):
    conteo_hubs[hub] += 1

print("Ocupación de hubs:")
costo_fijo_total = 0
for j in range(n_hubs):
    if conteo_hubs[j] > 0:
        print(f"Hub {j+1}: {conteo_hubs[j]}/3 clientes - Costo fijo: {costos_fijos[j]}")
        costo_fijo_total += costos_fijos[j]
    else:
        print(f"Hub {j+1}: No usado")

costo_distancias = sum(distancias[i][hub] for i, hub in enumerate(mejor_solucion))
print(f"\nCosto total distancias: {costo_distancias}")
print(f"Costo total fijo:       {costo_fijo_total}")
print(f"COSTO TOTAL:            {costo_distancias + costo_fijo_total}")

print(f"\nCOMPARACIÓN CON INSTANCIA ALTA:")
print("=" * 50)
print(f"Instancia Media: {n_clients} clientes, {n_hubs} hubs, capacidad {capacidades[0]}")



### Código TOC – Instancia ALTA

In [ ]:

import numpy as np
import random
import statistics
import time
from tqdm import tqdm

distancias_alta = np.array([
    [6, 9, 8, 7, 5, 8, 6, 9],
    [7, 6, 7, 9, 6, 5, 7, 8],
    [8, 7, 6, 9, 8, 9, 7, 6],
    [7, 6, 9, 8, 6, 7, 9, 8],
    [6, 5, 8, 9, 7, 8, 9, 7],
    [9, 6, 8, 9, 7, 9, 6, 8],
    [8, 7, 9, 8, 7, 8, 6, 7],
    [6, 8, 7, 9, 9, 7, 8, 6],
    [7, 9, 10, 9, 8, 7, 6, 5],
    [6, 8, 9, 7, 8, 9, 7, 8],
    [7, 6, 7, 8, 6, 8, 9, 7],
    [8, 9, 8, 9, 7, 8, 6, 5],
    [9, 8, 7, 6, 5, 6, 8, 9],
    [6, 9, 7, 8, 6, 7, 9, 7],
    [7, 9, 6, 8, 9, 7, 6, 8],
    [8, 6, 7, 9, 8, 9, 7, 6],
    [6, 8, 9, 7, 6, 5, 6, 7],
    [9, 7, 8, 9, 7, 8, 9, 8],
    [6, 9, 7, 8, 7, 6, 9, 8],
    [7, 8, 6, 9, 7, 8, 6, 9],
    [9, 7, 6, 9, 8, 9, 7, 6],
    [8, 9, 8, 7, 6, 5, 6, 8],
    [7, 6, 8, 7, 8, 9, 7, 5],
    [8, 5, 7, 9, 8, 7, 6, 6]
])
costos_fijos_alta = [22, 25, 18, 20, 19, 24, 21, 23]
capacidades_alta  = [4, 4, 4, 4, 4, 4, 4, 4]
D_MAX = 8
n_clients = distancias_alta.shape[0]
n_hubs = distancias_alta.shape[1]
dim = n_clients

def evaluar(asignacion):
    conteo = [0] * n_hubs
    usado = [0] * n_hubs
    costo = 0
    for c, j in enumerate(asignacion):
        if j < 0 or j >= n_hubs or distancias_alta[c][j] > D_MAX:
            return float('inf')
        conteo[j] += 1
        usado[j] = 1
        if conteo[j] > capacidades_alta[j]:
            return float('inf')
        costo += distancias_alta[c][j]
    for j in range(n_hubs):
        if usado[j]:
            costo += costos_fijos_alta[j]
    return costo

POP_SIZE = 20
MAX_ITERS = 30
NT = 4  # Thunderstorms
NO = 1  # Tornado
DIM = n_clients

def generar_individuo_factible():
    individuo = []
    for c in range(n_clients):
        opciones = [j for j in range(n_hubs) if distancias_alta[c][j] <= D_MAX]
        if opciones:
            individuo.append(random.choice(opciones))
        else:
            return None
    return individuo

def init_population():
    poblacion = []
    while len(poblacion) < POP_SIZE:
        ind = generar_individuo_factible()
        if ind is not None and evaluar(ind) < float('inf'):
            poblacion.append(np.array(ind))
    return poblacion

def clip(sol):
    return np.clip(sol, 0, n_hubs - 1).astype(int)


def toc_optimizer():
    pop = init_population()
    fitness = [evaluar(ind) for ind in pop]

    for t in range(MAX_ITERS):
        indices_sorted = np.argsort(fitness)
        pop = [pop[i] for i in indices_sorted]
        fitness = [fitness[i] for i in indices_sorted]

        tornado = pop[0]
        thunderstorms = pop[1:NT + 1]
        windstorms = pop[NT + 1:]

        new_pop = [tornado.copy()] + [ts.copy() for ts in thunderstorms]

        for w in windstorms:
            leader = random.choice([tornado] + thunderstorms)
            new_w = w.copy()
            for i in range(DIM):
                opciones = [j for j in range(n_hubs) if distancias_alta[i][j] <= D_MAX]
                if not opciones:
                    new_w[i] = -1
                elif random.random() < 0.5:
                    diff = leader[i] - w[i]
                    move = int(np.sign(diff)) if diff != 0 else 0
                    candidato = w[i] + move
                    new_w[i] = candidato if candidato in opciones else random.choice(opciones)
                else:
                    new_w[i] = random.choice(opciones)

            new_w = clip(new_w)
            if -1 in new_w or evaluar(new_w) == float('inf'):
                new_pop.append(w)
            elif evaluar(new_w) < evaluar(w):
                new_pop.append(new_w)
            else:
                new_pop.append(w)

        pop = new_pop
        fitness = [evaluar(ind) for ind in pop]

    best_fit = min(fitness)
    best_sol = pop[np.argmin(fitness)]
    return best_fit, best_sol


print(" Ejecutando TOC Algorithm - 30 corridas...")
print("=" * 50)

# Medir tiempo total de ejecución
tiempo_inicio = time.time()

resultados = []
mejores_soluciones = []
tiempos_individuales = []
print("\nRESULTADOS POR CORRIDA:")
print("-" * 30)

for corrida in range(30):
    # Medir tiempo de cada corrida individual
    inicio_corrida = time.time()
    val, sol = toc_optimizer()
    fin_corrida = time.time()

    tiempo_corrida = fin_corrida - inicio_corrida
    tiempos_individuales.append(tiempo_corrida)

    resultados.append(val)
    mejores_soluciones.append(sol)
    print(f"Corrida {corrida + 1:2d}: {val}")

    if (corrida + 1) % 10 == 0:
        print(f"Completadas {corrida + 1}/30 corridas... (Tiempo promedio por corrida: {np.mean(tiempos_individuales):.3f}s)")

# Tiempo total transcurrido
tiempo_fin = time.time()
tiempo_total = tiempo_fin - tiempo_inicio

print("\n RESULTADOS TOC - ESTADÍSTICAS:")
print("=" * 50)
print(f" Mejor resultado:        {min(resultados)}")
print(f" Peor resultado:         {max(resultados)}")
print(f" Promedio:               {np.mean(resultados):.2f}")
print(f" Mediana:                {np.median(resultados)}")
print(f" Desviación estándar:    {np.std(resultados):.2f}")
print(f" Rango intercuartílico:  {np.percentile(resultados, 75) - np.percentile(resultados, 25):.2f}")

print(f"\n⏱  TIEMPOS DE EJECUCIÓN:")
print("=" * 50)
print(f" Tiempo total:           {tiempo_total:.3f} segundos")
print(f" Tiempo promedio/corrida: {np.mean(tiempos_individuales):.3f} segundos")
print(f" Tiempo más rápido:      {min(tiempos_individuales):.3f} segundos")
print(f" Tiempo más lento:       {max(tiempos_individuales):.3f} segundos")
print(f" Desv. std tiempo:       {np.std(tiempos_individuales):.3f} segundos")

# Mostrar la mejor solución encontrada
mejor_indice = np.argmin(resultados)
mejor_solucion = mejores_soluciones[mejor_indice]
print(f"\nMEJOR SOLUCIÓN ENCONTRADA (Costo: {min(resultados)}):")
print("=" * 50)
print("Cliente → Hub asignado:")
for i, hub in enumerate(mejor_solucion):
    print(f"Cliente {i+1:2d} → Hub {hub+1} (distancia: {distancias_alta[i][hub]})")

# Verificar factibilidad de la mejor solución
print(f"\n🔍 VERIFICACIÓN DE LA MEJOR SOLUCIÓN:")
print("=" * 50)
conteo_hubs = [0] * n_hubs
for cliente, hub in enumerate(mejor_solucion):
    conteo_hubs[hub] += 1

print("Ocupación de hubs:")
costo_fijo_total = 0
for j in range(n_hubs):
    if conteo_hubs[j] > 0:
        print(f"Hub {j+1}: {conteo_hubs[j]}/4 clientes - Costo fijo: {costos_fijos_alta[j]}")
        costo_fijo_total += costos_fijos_alta[j]
    else:
        print(f"Hub {j+1}: No usado")

costo_distancias = sum(distancias_alta[i][hub] for i, hub in enumerate(mejor_solucion))
print(f"\nCosto total distancias: {costo_distancias}")
print(f"Costo total fijo:       {costo_fijo_total}")
print(f"COSTO TOTAL:            {costo_distancias + costo_fijo_total}")


### MiniZinc

In [ ]:
import subprocess
import time

def resolver_instancia(nombre_dzn):
    print(f"\n🔎 Resolviendo instancia: {nombre_dzn}")

    # Medir tiempo
    inicio = time.time()
    resultado = subprocess.run(["minizinc", "modelo_hubs.mzn", nombre_dzn], capture_output=True, text=True)
    fin = time.time()

    tiempo_total = fin - inicio

    print(f"⏱️ Tiempo: {tiempo_total:.2f} segundos")
    print(resultado.stdout.strip())

    if resultado.stderr:
        print("⚠️ Error:", resultado.stderr)

    return tiempo_total

# Ejecutar cada instancia y guardar tiempos
tiempos = {}
tiempos["baja"] = resolver_instancia("instancia_baja.dzn")
tiempos["media"] = resolver_instancia("instancia_media.dzn")

# Resumen final
print(f"\n📊 RESUMEN DE TIEMPOS:")
for instancia, tiempo in tiempos.items():
    print(f"  {instancia}: {tiempo:.2f}s")


🔎 Resolviendo instancia: instancia_baja.dzn
⏱️ Tiempo: 0.14 segundos
Asignación: [1, 2, 2, 2, 3, 1]
Costo total: 60
Distancia total: 29
----------

🔎 Resolviendo instancia: instancia_media.dzn
⏱️ Tiempo: 0.11 segundos
Asignación: [5, 3, 4, 3, 1, 1, 4, 1, 4, 3, 5, 5]
Costo total: 79
Distancia total: 75
----------

📊 RESUMEN DE TIEMPOS:
  baja: 0.14s
  media: 0.11s


In [ ]:
print("\n" + "="*50)
print("MÉTODO 4: SOLUCIÓN HEURÍSTICA EN PYTHON (BACKUP)")
print("="*50)

# Si MiniZinc falla, implementamos una heurística simple
import numpy as np

def solucion_heuristica():
    # Datos del problema
    dist_matrix = np.array([
        [6,9,8,7,5,8,6,9], [5,6,7,9,8,6,7,8], [8,7,9,6,9,8,9,7], [7,5,6,9,7,9,8,8],
        [6,9,6,7,8,9,8,8], [9,6,8,9,6,7,7,6], [8,7,9,8,9,6,7,7], [7,8,9,6,7,7,6,9],
        [6,5,8,7,6,9,7,8], [7,9,6,8,9,8,6,7], [8,6,7,9,7,6,8,6], [6,7,9,7,6,9,8,8],
        [7,9,6,8,7,8,9,7], [9,6,8,7,8,9,6,9], [7,8,9,8,7,8,9,7], [5,6,8,9,6,8,7,8],
        [6,9,7,6,9,8,8,6], [6,8,7,7,6,9,7,9], [7,9,6,9,8,6,9,8], [6,7,8,7,8,6,7,9],
        [8,9,7,8,9,6,7,6], [7,9,6,7,6,9,6,8], [8,6,7,9,7,6,8,7], [5,8,7,9,7,8,7,6]
    ])

    capacidades = [4, 4, 4, 4, 4, 4, 4, 4]
    costos_fijos = [22, 25, 18, 20, 19, 24, 21, 23]
    dmax = 8

    # Heurística greedy: asignar cada cliente al hub más barato disponible
    asignacion = [-1] * 24
    ocupacion_hubs = [0] * 8

    # Ordenar hubs por costo (más baratos primero)
    hubs_ordenados = sorted(range(8), key=lambda x: costos_fijos[x])

    for cliente in range(24):
        asignado = False
        # Intentar asignar al hub más barato con capacidad
        for hub in hubs_ordenados:
            if (ocupacion_hubs[hub] < capacidades[hub] and
                dist_matrix[cliente][hub] <= dmax):
                asignacion[cliente] = hub + 1  # MiniZinc usa índices 1-based
                ocupacion_hubs[hub] += 1
                asignado = True
                break

        if not asignado:
            # Buscar cualquier hub disponible
            for hub in range(8):
                if (ocupacion_hubs[hub] < capacidades[hub] and
                    dist_matrix[cliente][hub] <= dmax):
                    asignacion[cliente] = hub + 1
                    ocupacion_hubs[hub] += 1
                    asignado = True
                    break

    # Calcular costos
    hubs_usados = set(asignacion)
    costo_fijo = sum(costos_fijos[h-1] for h in hubs_usados)
    distancia_total = sum(dist_matrix[i][asignacion[i]-1] for i in range(24))

    print("=== SOLUCIÓN HEURÍSTICA ===")
    print(f"Asignación: {asignacion}")
    print(f"Costo fijo: {costo_fijo}")
    print(f"Distancia total: {distancia_total}")
    print(f"TOTAL: {costo_fijo + distancia_total}")
    print(f"Hubs utilizados: {sorted(hubs_usados)}")

    return asignacion, costo_fijo + distancia_total

# Ejecutar heurística como backup

solucion_heuristica()

print("\n" + "="*50)
print("="*50)
print("4. Heurística Python: Solución aproximada instantánea")


In [ ]:
import os
print(os.getcwd())

/content
